In [ ]:
import pandas as pd
import numpy as np
import os
import glob
import time
import logging
import torch
import threading
from concurrent.futures import ThreadPoolExecutor
from queue import Queue
from datetime import datetime
from tqdm import tqdm
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from transformers import BertTokenizer, BertForSequenceClassification

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'news_sentiment_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Constants
MAX_TEXT_LENGTH = 60000  # Match the same limit as in original code
BATCH_SIZE = 20  # Increased batch size for news articles (which are shorter)
MAX_WORKERS = min(6, os.cpu_count())  # Use a reasonable number of threads
MIN_TITLE_LENGTH = 5  # Minimum title length for meaningful sentiment analysis

# Initialize models - shared across threads
vader_analyzer = None
finbert_tokenizer = None
finbert_model = None
device = None

def initialize_models():
    """Initialize the sentiment models - only needs to be done once"""
    global vader_analyzer, finbert_tokenizer, finbert_model, device
    
    try:
        # Initialize VADER
        vader_analyzer = SentimentIntensityAnalyzer()
        
        # Initialize FinBERT
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        logger.info(f"Using device: {device}")
        
        finbert_tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
        finbert_model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert').to(device)
        finbert_model.eval()  # Set to evaluation mode
        
        logger.info("Models initialized successfully!")
    except Exception as e:
        logger.error(f"Error initializing models: {e}")
        raise

# Function to prepare news text for sentiment analysis
def prepare_news_text(title):
    """
    Prepare news text for sentiment analysis
    For news, we primarily use the title as that's what's available
    """
    # Handle NaN or None values
    if pd.isna(title) or title is None:
        title = ''
    
    # If title is empty or too short, skip
    if not title or len(title) < MIN_TITLE_LENGTH:
        return None
    
    return title

# VADER sentiment analysis function
def analyze_with_vader(text):
    """
    Analyze sentiment using VADER
    """
    global vader_analyzer
    
    try:
        # Truncate text if too long
        if text and len(text) > MAX_TEXT_LENGTH:
            text = text[:MAX_TEXT_LENGTH]
        
        if not text or not isinstance(text, str):
            return {
                'vader_compound': np.nan,
                'vader_pos': np.nan,
                'vader_neg': np.nan,
                'vader_neu': np.nan
            }
        
        # Get sentiment scores
        scores = vader_analyzer.polarity_scores(text)
        
        return {
            'vader_compound': scores['compound'],
            'vader_pos': scores['pos'],
            'vader_neg': scores['neg'],
            'vader_neu': scores['neu']
        }
    except Exception as e:
        logger.error(f"Error in VADER analysis: {e}")
        return {
            'vader_compound': np.nan,
            'vader_pos': np.nan,
            'vader_neg': np.nan,
            'vader_neu': np.nan
        }

# FinBERT sentiment analysis function
def analyze_with_finbert(text):
    """
    Analyze sentiment using FinBERT with chunking for long texts
    """
    global finbert_tokenizer, finbert_model, device
    
    try:
        if not text or not isinstance(text, str):
            return {
                'finbert_positive': np.nan,
                'finbert_negative': np.nan,
                'finbert_neutral': np.nan,
                'finbert_sentiment': np.nan
            }
        
        # Truncate text if too long
        if len(text) > MAX_TEXT_LENGTH:
            text = text[:MAX_TEXT_LENGTH]
        
        # Tokenize and split into chunks of 512 tokens
        encoded_input = finbert_tokenizer(text, 
                                return_tensors='pt', 
                                max_length=512, 
                                truncation=True, 
                                padding=True, 
                                return_overflowing_tokens=True)
        
        # Get number of chunks
        num_chunks = encoded_input['input_ids'].size(0)
        
        if num_chunks == 0:
            return {
                'finbert_positive': np.nan,
                'finbert_negative': np.nan,
                'finbert_neutral': np.nan,
                'finbert_sentiment': np.nan
            }
        
        # Process all chunks in one batch (or in smaller batches if there are too many)
        all_probs = []
        batch_size = 8  # Process chunks in batches of 8 if there are many
        
        for i in range(0, num_chunks, batch_size):
            batch_input_ids = encoded_input['input_ids'][i:i+batch_size].to(device)
            batch_attention_mask = encoded_input['attention_mask'][i:i+batch_size].to(device)
            
            with torch.no_grad():
                outputs = finbert_model(batch_input_ids, attention_mask=batch_attention_mask)
                logits = outputs.logits
                probs = torch.softmax(logits, dim=1)
                all_probs.append(probs)
        
        # Combine all batches
        combined_probs = torch.cat(all_probs, dim=0)
        
        # Average probabilities across all chunks
        avg_probs = combined_probs.mean(dim=0).cpu().numpy()
        
        # Map to sentiment categories (FinBERT order: positive, negative, neutral)
        return {
            'finbert_positive': float(avg_probs[0]),
            'finbert_negative': float(avg_probs[1]),
            'finbert_neutral': float(avg_probs[2]),
            'finbert_sentiment': float(avg_probs[0] - avg_probs[1])  # pos - neg as overall score
        }
    except Exception as e:
        logger.error(f"Error in FinBERT analysis: {e}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        return {
            'finbert_positive': np.nan,
            'finbert_negative': np.nan,
            'finbert_neutral': np.nan,
            'finbert_sentiment': np.nan
        }

# Process a single row and put the results in the shared results list
def process_row(args):
    idx, title, results_list = args
    
    try:
        # Prepare text
        text = prepare_news_text(title)
        if not text:
            results_list.append((idx, None, None))
            return
        
        # Process with VADER
        vader_results = analyze_with_vader(text)
        
        # Process with FinBERT
        finbert_results = analyze_with_finbert(text)
        
        # Add to results
        results_list.append((idx, vader_results, finbert_results))
    except Exception as e:
        logger.error(f"Error processing row {idx}: {e}")
        results_list.append((idx, None, None))

# Find ticker file paths
def find_news_files(ticker=None):
    """
    Find news files for a specific ticker or all tickers
    """
    base_path = "sentiment_results/news/top30"
    
    if ticker:
        # Find file for specific ticker
        file_path = f"{base_path}/news_sentiment_{ticker}.csv"
        if os.path.exists(file_path):
            return [file_path]
        else:
            logger.warning(f"No file found for ticker {ticker}")
            return []
    else:
        # Find all ticker files
        pattern = f"{base_path}/news_sentiment_*.csv"
        files = glob.glob(pattern)
        if files:
            logger.info(f"Found {len(files)} ticker files")
            return files
        else:
            logger.warning("No ticker files found")
            return []

# Process a file using ThreadPoolExecutor
def process_file(input_file, output_file):
    """
    Process a file using ThreadPoolExecutor for parallelism
    """
    try:
        logger.info(f"Loading input file: {input_file}")
        
        # Check if input file exists
        if not os.path.exists(input_file):
            logger.error(f"Input file {input_file} not found.")
            return None
        
        # Load news data
        news_df = pd.read_csv(input_file)
        logger.info(f"Loaded {len(news_df)} rows from {input_file}")
        
        # Check if 'title' column exists
        if 'title' not in news_df.columns:
            logger.error("News data missing required column: title")
            return None
        
        # Initialize models if not already initialized
        if vader_analyzer is None:
            initialize_models()
        
        # Shared results list - threads will append to this
        all_results = []
        
        # Prepare arguments for each row (idx, title, results_list)
        row_args = [(idx, row['title'], all_results) for idx, row in news_df.iterrows()]
        
        # Process in batches for better progress tracking
        total_rows = len(row_args)
        batches = [row_args[i:i+BATCH_SIZE] for i in range(0, total_rows, BATCH_SIZE)]
        
        logger.info(f"Processing {len(batches)} batches with {MAX_WORKERS} threads")
        
        # Process all batches
        for batch_idx, batch in enumerate(tqdm(batches, desc="Processing batches")):
            # Process batch with ThreadPoolExecutor
            with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
                list(tqdm(executor.map(process_row, batch), 
                         total=len(batch), 
                         desc=f"Batch {batch_idx+1}/{len(batches)}"))
            
            # Save intermediate results after each batch
            temp_df = news_df.copy()
            for idx, vader_result, finbert_result in all_results:
                if vader_result:
                    for key, value in vader_result.items():
                        temp_df.at[idx, key] = value
                if finbert_result:
                    for key, value in finbert_result.items():
                        temp_df.at[idx, key] = value
            
            temp_output = output_file.replace('.csv', f'_temp_{batch_idx+1}.csv')
            os.makedirs(os.path.dirname(temp_output), exist_ok=True)
            temp_df.to_csv(temp_output, index=False)
            logger.info(f"Saved intermediate results to {temp_output} after batch {batch_idx+1}")
        
        # Update dataframe with all results
        output_df = news_df.copy()
        
        # Add new sentiment scores
        for idx, vader_result, finbert_result in all_results:
            if vader_result:
                for key, value in vader_result.items():
                    output_df.at[idx, key] = value
            if finbert_result:
                for key, value in finbert_result.items():
                    output_df.at[idx, key] = value
        
        # Save final results
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        output_df.to_csv(output_file, index=False)
        logger.info(f"Saved final results to {output_file}")
        
        # Clean up temporary files (no need to ask)
        temp_files = [f for f in os.listdir(os.path.dirname(output_file)) 
                     if os.path.basename(f).startswith(os.path.basename(output_file).split('.')[0] + '_temp_')]
        
        for temp_file in temp_files:
            temp_path = os.path.join(os.path.dirname(output_file), temp_file)
            try:
                os.remove(temp_path)
                logger.info(f"Removed {temp_file}")
            except Exception as e:
                logger.error(f"Could not remove {temp_file}: {e}")
        
        return output_df
    
    except Exception as e:
        logger.error(f"Error processing file: {e}")
        import traceback
        logger.error(f"Traceback: {traceback.format_exc()}")
        return None

def main():
    print("\n=== ENHANCING NEWS SENTIMENT ANALYSIS WITH VADER AND FINBERT ===")
    
    # Ask user for tickers to process
    tickers_input = input("Which tickers would you like to process? (Enter comma-separated values, e.g., 'AAPL,MSFT' or 'all' for all tickers): ")
    
    if tickers_input.lower() == 'all':
        # Find all ticker files
        news_files = find_news_files()
        if not news_files:
            print("No ticker files found. Please check the path: sentiment_results/news/top30/")
            return
    else:
        tickers = [ticker.strip().upper() for ticker in tickers_input.split(',')]
        news_files = []
        for ticker in tickers:
            ticker_files = find_news_files(ticker)
            if ticker_files:
                news_files.extend(ticker_files)
            else:
                print(f"No file found for ticker {ticker}")
    
    if not news_files:
        print("No files to process.")
        return
    
    # Process each ticker file without asking for confirmation
    for news_file in news_files:
        ticker = os.path.basename(news_file).split('_')[-1].split('.')[0]
        
        # Define output file path
        output_file = news_file.replace('.csv', '_with_vader_finbert.csv')
        
        print(f"\n=== PROCESSING TICKER: {ticker} ===")
        print(f"Input file: {news_file}")
        print(f"Output file: {output_file}")
        
        # Start timing
        start_time = time.time()
        
        # Process the file
        process_file(news_file, output_file)
        
        # End timing
        elapsed_time = time.time() - start_time
        
        print(f"Completed enhancement for ticker {ticker}")
        print(f"Processing time: {elapsed_time:.2f} seconds")

if __name__ == "__main__":
    # Check if required libraries are installed
    try:
        import nltk
        nltk.data.find('vader_lexicon')
    except (ImportError, LookupError):
        print("NLTK VADER lexicon not found. Installing...")
        import nltk
        nltk.download('vader_lexicon')
    
    try:
        import transformers
    except ImportError:
        print("Transformers library not found. Please install it with:")
        print("pip install transformers")
        if input("Continue anyway? (y/n): ").lower() != 'y':
            import sys
            sys.exit(1)
    
    try:
        import torch
    except ImportError:
        print("PyTorch not found. Please install it with:")
        print("pip install torch")
        if input("Continue anyway? (y/n): ").lower() != 'y':
            import sys
            sys.exit(1)
    
    # Run main function
    main()

/home/nawa/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-22 00:32:52.771367: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-22 00:32:52.783021: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745299972.795159   80070 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745299972.798913   80070 cuda_blas.cc:1407] Unable to re

NLTK VADER lexicon not found. Installing...

=== ENHANCING NEWS SENTIMENT ANALYSIS WITH VADER AND FINBERT ===


Which tickers would you like to process? (Enter comma-separated values, e.g., 'AAPL,MSFT' or 'all' for all tickers):  all


2025-04-22 00:32:58,968 - INFO - Found 30 ticker files
2025-04-22 00:32:58,969 - INFO - Loading input file: sentiment_results/news/top30/news_sentiment_CRM.csv
2025-04-22 00:32:58,989 - INFO - Loaded 2257 rows from sentiment_results/news/top30/news_sentiment_CRM.csv
2025-04-22 00:32:59,043 - INFO - Using device: cuda



=== PROCESSING TICKER: CRM ===
Input file: sentiment_results/news/top30/news_sentiment_CRM.csv
Output file: sentiment_results/news/top30/news_sentiment_CRM_with_vader_finbert.csv


...

Batch 1/109: 100%|██████████| 20/20 [00:00<00:00, 157.35it/s]
2025-04-22 00:35:06,653 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ORCL_with_vader_finbert_temp_1.csv after batch 1
Batch 2/109: 100%|██████████| 20/20 [00:00<00:00, 175.67it/s]
2025-04-22 00:35:06,843 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ORCL_with_vader_finbert_temp_2.csv after batch 2
Batch 3/109: 100%|██████████| 20/20 [00:00<00:00, 162.13it/s]
2025-04-22 00:35:07,087 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ORCL_with_vader_finbert_temp_3.csv after batch 3
Batch 4/109: 100%|██████████| 20/20 [00:00<00:00, 181.50it/s]
2025-04-22 00:35:07,275 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ORCL_with_vader_finbert_temp_4.csv after batch 4
Batch 5/109: 100%|██████████| 20/20 [00:00<00:00, 161.88it/s]
2025-04-22 00:35:07,479 - INFO - Saved intermediate results to sen

Completed enhancement for ticker ORCL
Processing time: 30.63 seconds

=== PROCESSING TICKER: ACN ===
Input file: sentiment_results/news/top30/news_sentiment_ACN.csv
Output file: sentiment_results/news/top30/news_sentiment_ACN_with_vader_finbert.csv


Batch 1/52: 100%|██████████| 20/20 [00:00<00:00, 177.15it/s]
2025-04-22 00:35:37,175 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ACN_with_vader_finbert_temp_1.csv after batch 1
Batch 2/52: 100%|██████████| 20/20 [00:00<00:00, 143.48it/s]
2025-04-22 00:35:37,344 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ACN_with_vader_finbert_temp_2.csv after batch 2
Batch 3/52: 100%|██████████| 20/20 [00:00<00:00, 144.61it/s]
2025-04-22 00:35:37,535 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ACN_with_vader_finbert_temp_3.csv after batch 3
Batch 4/52: 100%|██████████| 20/20 [00:00<00:00, 129.37it/s]
2025-04-22 00:35:37,749 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_ACN_with_vader_finbert_temp_4.csv after batch 4
Batch 5/52: 100%|██████████| 20/20 [00:00<00:00, 130.48it/s]
2025-04-22 00:35:37,963 - INFO - Saved intermediate results to sentiment_re

Completed enhancement for ticker ACN
Processing time: 12.39 seconds

=== PROCESSING TICKER: TSLA ===
Input file: sentiment_results/news/top30/news_sentiment_TSLA.csv
Output file: sentiment_results/news/top30/news_sentiment_TSLA_with_vader_finbert.csv


2025-04-22 00:35:49,727 - INFO - Processing 826 batches with 6 threads
Batch 1/826: 100%|██████████| 20/20 [00:00<00:00, 163.29it/s]
2025-04-22 00:35:50,083 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_TSLA_with_vader_finbert_temp_1.csv after batch 1
Batch 2/826: 100%|██████████| 20/20 [00:00<00:00, 156.77it/s]
2025-04-22 00:35:50,444 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_TSLA_with_vader_finbert_temp_2.csv after batch 2
Batch 3/826: 100%|██████████| 20/20 [00:00<00:00, 153.11it/s]
2025-04-22 00:35:50,809 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_TSLA_with_vader_finbert_temp_3.csv after batch 3
Batch 4/826: 100%|██████████| 20/20 [00:00<00:00, 153.03it/s]
2025-04-22 00:35:51,177 - INFO - Saved intermediate results to sentiment_results/news/top30/news_sentiment_TSLA_with_vader_finbert_temp_4.csv after batch 4
Batch 5/826: 100%|██████████| 20/20 [00:00<00:00, 139.56i